# Eat Safe, Love

## Notebook Set Up

In [7]:
# Import dependencies
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [8]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [9]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [10]:
# review the collections in our database
collections = db.list_collection_names()


print("Collections in the 'uk_food' database:")
for collection in collections:
    print(f"- {collection}")

Collections in the 'uk_food' database:
- establishments


In [11]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [12]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"Number of establishments with a hygiene score of 20: {count}")
# Display the first document in the results using pprint
first_result = establishments.find_one(query)
print("First establishment with a hygiene score of 20:")
pprint(first_result)

Number of establishments with a hygiene score of 20: 82
First establishment with a hygiene score of 20:
{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': 0,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('66b8900e1d6b55af2db63a23'),
 'geocode': {'latitude': Decimal128('50.769705'),
             'longitude': Decimal128('0.27694')},
 'links': [{'

In [28]:
# Convert the result to a Pandas DataFrame
results = list(establishments.find({"scores.Hygiene": 20}))
df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))

Number of rows in the DataFrame: 82
First 10 rows of the DataFrame:
                        _id   FHRSID  ChangesByServerID  \
0  66b8900e1d6b55af2db63a23   110681                  0   
1  66b8900e1d6b55af2db63da2   612039                  0   
2  66b8900f1d6b55af2db640ad   730933                  0   
3  66b8900f1d6b55af2db6429c   172735                  0   
4  66b8900f1d6b55af2db642aa   172953                  0   
5  66b8900f1d6b55af2db64c48   512854                  0   
6  66b8900f1d6b55af2db64e6a  1537089                  0   
7  66b8900f1d6b55af2db663c9   155648                  0   
8  66b8900f1d6b55af2db667da  1012883                  0   
9  66b8900f1d6b55af2db66fee   644109                  0   

  LocalAuthorityBusinessID               BusinessName  \
0                     4029        The Chase Rest Home   
1                1970/FOOD                 Brenalwood   
2                1698/FOOD              Melrose Hotel   
3             PI/000023858              Seaford Pizza 

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [14]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {"LocalAuthorityName": "London", "RatingValue": {"$gte": 4}}
# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"Number of establishments with London as the Local Authority and RatingValue >= 4: {count}")
# Display the first document in the results using pprint
first_result = establishments.find_one(query)
print("First establishment with London as the Local Authority and RatingValue >= 4:")
pprint(first_result)

Number of establishments with London as the Local Authority and RatingValue >= 4: 0
First establishment with London as the Local Authority and RatingValue >= 4:
None


In [27]:
# Convert the result to a Pandas DataFrame
results = list(establishments.find({"LocalAuthorityName": "London", "RatingValue": {"$gte": 4}}))
df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))

Number of rows in the DataFrame: 0
First 10 rows of the DataFrame:
Empty DataFrame
Columns: []
Index: []


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [45]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

# Print the results
degree_search = 0.01  

query = {
    "RatingValue": 5,
    "geocode.latitude": {"$gte": penang_latitude - degree_search, "$lte": penang_latitude + degree_search},
    "geocode.longitude": {"$gte": penang_longitude - degree_search, "$lte": penang_longitude + degree_search}
}

sort = [("scores.Hygiene", 1)]
limit = 5
top_5_near_penang = list(establishments.find(query).sort(sort).limit(limit))
pprint(top_5_near_penang)

[{'AddressLine1': '64 Plumstead Common Road',
  'AddressLine2': '',
  'AddressLine3': 'Plumstead',
  'AddressLine4': 'Greenwich',
  'BusinessName': 'Taxcos',
  'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  'ChangesByServerID': 0,
  'Distance': 4647.467442824658,
  'FHRSID': 1419348,
  'LocalAuthorityBusinessID': '14817',
  'LocalAuthorityCode': '511',
  'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
  'LocalAuthorityName': 'Greenwich',
  'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
  'NewRatingPending': False,
  'Phone': '',
  'PostCode': 'SE18 3RD',
  'RatingDate': '2022-03-28T00:00:00',
  'RatingKey': 'fhrs_5_en-gb',
  'RatingValue': 5,
  'RightToReply': '',
  'SchemeType': 'FHRS',
  '_id': ObjectId('66b8900f1d6b55af2db69551'),
  'geocode': {'latitude': Decimal128('51.4817810058594'),
              'longitude': Decimal128('0.0727310031652451')},
  'links': [{'href': 'http://api.ratings.food.gov.uk/establishments/1419348',
         

In [46]:
# Convert result to Pandas DataFrame
df = pd.DataFrame(top_5_near_penang)
print("Top 5 establishments near Penang Flavours:")
print(df)

Top 5 establishments near Penang Flavours:
                        _id   FHRSID  ChangesByServerID  \
0  66b8900f1d6b55af2db69551  1419348                  0   
1  66b8900f1d6b55af2db6959a   871609                  0   
2  66b8900f1d6b55af2db69535   942472                  0   
3  66b8900f1d6b55af2db69534  1380581                  0   
4  66b8900f1d6b55af2db6954e   695511                  0   

  LocalAuthorityBusinessID                         BusinessName  \
0                    14817                               Taxcos   
1             PI/000198274                  Steve's Fruit & Veg   
2             PI/000202326  St Margaret's C of E Primary School   
3                    14428                  Bal Krishna Nursery   
4             PI/000187355                Avante Care & Support   

                BusinessType  BusinessTypeID  \
0    Restaurant/Cafe/Canteen               1   
1          Retailers - other            4613   
2  School/college/university            7845   
3      

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [47]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

pipeline = [
    {
        "$match": {"scores.Hygiene": 0}  
    },
    {
        "$group": {                      
            "_id": "$LocalAuthorityName",
            "count": {"$sum": 1}         
        }
    },
    {
        "$sort": {"count": -1}           
    }
]

# Print the number of documents in the result
results = list(establishments.aggregate(pipeline))
print(f"Number of documents in the result: {len(results)}")

# Print the first 10 results
print("First 10 results:")
pprint(results[:10])

Number of documents in the result: 55
First 10 results:
[{'_id': 'Thanet', 'count': 2260},
 {'_id': 'Greenwich', 'count': 1764},
 {'_id': 'Maidstone', 'count': 1426},
 {'_id': 'Newham', 'count': 1422},
 {'_id': 'Swale', 'count': 1372},
 {'_id': 'Chelmsford', 'count': 1360},
 {'_id': 'Medway', 'count': 1344},
 {'_id': 'Bexley', 'count': 1214},
 {'_id': 'Southend-On-Sea', 'count': 1172},
 {'_id': 'Tendring', 'count': 1084}]


In [48]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))


Number of rows in the DataFrame: 55
First 10 rows of the DataFrame:
               _id  count
0           Thanet   2260
1        Greenwich   1764
2        Maidstone   1426
3           Newham   1422
4            Swale   1372
5       Chelmsford   1360
6           Medway   1344
7           Bexley   1214
8  Southend-On-Sea   1172
9         Tendring   1084
